In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [5]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
#from pyimagesearch.lenet import LeNet
from imutils import paths

In [6]:
# import the necessary packages
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K

In [7]:
class LeNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model
		model = Sequential()
		inputShape = (height, width, depth)

		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)

		# first set of CONV => RELU => POOL layers
		model.add(Conv2D(64, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# second set of CONV => RELU => POOL layers
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
		# third set of CONV => RELU => POOL layers
		model.add(Conv2D(256, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
              
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1000))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

In [9]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = 50
INIT_LR = 1e-3
BS = 32

In [10]:
# init the image suffix, yset, and image list
print("[INFO] loading images...")
suffix = '.jpg'
img_list = []
yset = []

[INFO] loading images...


In [11]:
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

In [18]:
# Testing Cells
csv_file = 'balthus.csv'

In [19]:
# use csv file to grab images/labels
df = pd.read_csv(csv_file)
nameCol = df['#img']
predCol = df['Genre']

In [20]:
# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])

In [21]:
# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

In [22]:
# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i

In [23]:
label_yval

{'cityscape': 4,
 'genre painting': 3,
 'landscape': 7,
 'literary painting': 9,
 'nude painting (nu)': 5,
 'portrait': 2,
 'self-portrait': 6,
 'sketch and study': 8,
 'still life': 0,
 'symbolic painting': 1}

In [24]:
len(labels)

96

In [25]:
label_yval[labels[0]]

3

In [26]:
# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])

In [27]:
len(yset)

96

In [28]:
# remove text and leave fabric cave number for labels and zero index
#for i in range (0,len(yset)):
#    yset[i] = int(re.sub("\D", "", yset[i]))
#    yset[i] = yset[i]-1

In [29]:
# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join("./images3/", base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

In [30]:
Y = []
X = []

In [31]:
# Shuffle the data
p = np.random.permutation(len(yset))

In [32]:
for i in range(0,len(yset)):
    Y.append(yset[p[i]])
    X.append(img_list[p[i]])

In [33]:
# split the test and training set 75:25
split = int(len(X)*(.75))
xtrain = X[:split]
xtest = X[split:]
ytrain = Y[:split]
ytest = Y[split:]

In [34]:
# transform to arrays
trainX = np.array(xtrain, dtype="float")/225.0
testX = np.array(xtest, dtype ="float")/225.0

ytrain = np.array(ytrain)
ytest = np.array(ytest)

In [35]:
trainX.shape

(72, 28, 28, 3)

In [36]:
# parsed Y data containers
trainY = []
testY = []

In [37]:
# convert labels from int to vectors
trainY = np_utils.to_categorical(ytrain,12)
testY = np_utils.to_categorical(ytest,12)

In [38]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

In [39]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=12)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics=["accuracy"])

[INFO] compiling model...


In [40]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/50
3/2 [=============================================] - 2s 513ms/step - loss: 2.3651 - acc: 0.3219 - val_loss: 1.8711 - val_acc: 0.1667
Epoch 2/50
3/2 [=============================================] - 1s 282ms/step - loss: 2.2900 - acc: 0.0769 - val_loss: 2.1462 - val_acc: 0.0000e+00
Epoch 3/50
3/2 [=============================================] - 1s 288ms/step - loss: 2.0501 - acc: 0.2728 - val_loss: 1.9749 - val_acc: 0.4167
Epoch 4/50
3/2 [=============================================] - 1s 278ms/step - loss: 1.8552 - acc: 0.4147 - val_loss: 1.9119 - val_acc: 0.4167
Epoch 5/50
3/2 [=============================================] - 1s 285ms/step - loss: 1.8998 - acc: 0.4325 - val_loss: 1.8669 - val_acc: 0.4167
Epoch 6/50
3/2 [=============================================] - 1s 277ms/step - loss: 1.9415 - acc: 0.3041 - val_loss: 1.9189 - val_acc: 0.4167
Epoch 7/50
3/2 [=============================================] - 1s 281ms/step - loss: 1.8442 - acc

In [41]:
# plot the training loss and accuracy 
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Label Prediction")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('figure.png')

In [42]:
# Shows chart in a popup. Close to keep going
#chart = cv2.imread('figure.png',1)
#cv2.imshow('Results',chart)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [43]:
# Reshape original input data images for predicting
img_check = np.array(img_list, dtype ="float")/225.0

In [44]:
predictionsMade = []

In [45]:
preds = model.predict(img_check)

In [46]:
# Run all data through the prediction model that was created
for i in range (0,len(img_check)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)

In [47]:
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 

In [48]:
print("Accuracy: " + str(correct/len(yset)))

Accuracy: 0.4895833333333333


In [49]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

In [50]:
# Append the new column
df['#predictions'] = finalPred

In [51]:
# new file name
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(new_file, index=None)